# Оценка надежности заемщика банка в условиях неполноты информации

**Цель исследования**: оценка надежности заемщика банка в условиях неполноты исходной информации: наличия пропущенных данных и артефактов в предоставленных банком данных.

 - Заказчик — кредитный отдел банка. 
 - Входные данные от банка — статистика о платёжеспособности клиентов.
 - Объектом исследования является заёмщик банка.
 - Предмет исследования - надёжность заёмщика банка.


Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Задачи:**

- Разработка подхода к предобработке данных;
- Обработка пропущенных значений и артефактов;
- Обработка дубликатов данных;
- В рамках данной работы необходимо ответить на следующие вопросы:
    - Есть ли зависимость между наличием детей и возвратом кредита в срок?
    - Есть ли зависимость между семейным положением и возвратом кредита в срок?
    - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    - Как разные цели кредита влияют на его возврат в срок?
    

## 1. Изучение общей информации

In [39]:
import pandas as pd #loading the necessary libraries
import numpy as np
import pymorphy2
from collections import Counter
from nltk.corpus import stopwords
pd.options.mode.chained_assignment = None  # default='warn'
data = pd.read_csv('data.csv') #read the file
display(data.head(10))
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,"-8,437.67",42,высшее,0,женат / замужем,0,F,сотрудник,0,"253,875.64",покупка жилья
1,1,"-4,024.80",36,среднее,1,женат / замужем,0,F,сотрудник,0,"112,080.01",приобретение автомобиля
2,0,"-5,623.42",33,Среднее,1,женат / замужем,0,M,сотрудник,0,"145,885.95",покупка жилья
3,3,"-4,124.75",32,среднее,1,женат / замужем,0,M,сотрудник,0,"267,628.55",дополнительное образование
4,0,"340,266.07",53,среднее,1,гражданский брак,1,F,пенсионер,0,"158,616.08",сыграть свадьбу
5,0,-926.19,27,высшее,0,гражданский брак,1,M,компаньон,0,"255,763.57",покупка жилья
6,0,"-2,879.20",43,высшее,0,женат / замужем,0,F,компаньон,0,"240,525.97",операции с жильем
7,0,-152.78,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,"135,823.93",образование
8,2,"-6,929.87",35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,"95,856.83",на проведение свадьбы
9,0,"-2,188.76",41,среднее,1,женат / замужем,0,M,сотрудник,0,"144,425.94",покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Мы имеем данные с таблицей, состоящей из 12 столбцов. 
-	children — количество детей в семье
-	days_employed — общий трудовой стаж в днях
-	dob_years — возраст клиента в годах
-	education — уровень образования клиента
-	education_id — идентификатор уровня образования
-	family_status — семейное положение
-	family_status_id — идентификатор семейного положения
-	gender — пол клиента
-	income_type — тип занятости
-	debt — имел ли задолженность по возврату кредитов
-	total_income — ежемесячный доход
-	purpose — цель получения кредита


Названия столбцов не содержат пробелов, все знаки в одном регистре и на одном языке.

Есть категориальные и количественные данные. 

Есть пропуски в столбцах `'days_employed'`, `'total_income'`.

Значения в столбце `'days_employed'` представлены со знаком минус.

Типы столбцов соответствуют информации хранящихся в них.

Некоторые данные представлены в нижнем регистре, некоторые в верхнем. (пример: столбец `'education'` - среднее и СРЕДНЕЕ). Лучше перевести все в нижний регистр.

## 2. Предобработка данных

<a name='дубликаты'></a>
### Обработка дубликатов

In [40]:
data.duplicated().sum()

54

Явных дубликатов 54 строки.

Обычно дубликаты удаляются из датафрейма. Но я предлагаю взглянуть на них более пристально.

В наших данных может быть два разных человека, у которых бы совпадали данные по следующим столбцам: `children`, `dob_years`, `education`, `family_status`, `gender`, `income_type`, `debt`, `purpose` - это не те дубликаты которые надо удалять.

НО! те данные которые есть в столбцах `days_employed` и `total_income` вряд ли совпадут у двух разных клиентов. Посмотрим на значения столбцов `days_employed` и `total_income` для наших дубликатов.

In [41]:
print(data[data.duplicated()].days_employed.unique()) 
print(data[data.duplicated()].total_income.unique())

[nan]
[nan]


Таким образом у всех дубликатов отсутствуют значения в столбцах `days_employed` и `total_income`. Поэтому считаем их не повторами, а похожими людьми и оставляем в нашем датасете.

В процессе предобработки данных могут возникнут еще дубликаты (например после перевода значений в нижний регистр или заполнение пропущенных значений медианным или после лемматизации), но мы и в дальнейшем не будем удалять дубликаты, поскольку делаем предположение, что это просто клиенты с одинаковыми данными.

### Обработка невозможных значений и пропусков  в столбцах.

Часто предоставленные данные имеют пропуски, возникшие по тем или иным причинам, что влечёт за собой снижение эффективности применения статистических моделей. А также могут присутствовать артефакты(невозможные значения), которые в неисправленом виде могут построить неверную модель и привести к ошибкам. Проверим, есть ли такие в наших данных. 

In [42]:
print('        Доля пропусков(в процентах):')
print(data.isna().sum().apply(lambda x: round(x*100/len(data)))) # calculate the percentage of missing values for each column
print('Пропущенные значения в одних и тех же строках  - ', data[data.days_employed.isna()].equals(data[data.total_income.isna()])) # check if NaN values are in the same rows

        Доля пропусков(в процентах):
children             0
days_employed       10
dob_years            0
education            0
education_id         0
family_status        0
family_status_id     0
gender               0
income_type          0
debt                 0
total_income        10
purpose              0
dtype: int64
Пропущенные значения в одних и тех же строках  -  True


Уже на этопе изучения общей информации, мы заметили, что некоторые значения в столбцах не соответствуют данным которые должны там храниться. А также в двух столбцах: `days_employed` и `total_income` имеются пропущенные значения. Пропущенно около 10% значений, что достаточно много для того что бы не обращать на них внимания и удалить их. Проверено, если пропущенно значение об общем трудовом стаже, то и значение ежемесячного дохода тоже отсутствует. Есть предположение, что люди не захотели указывать эту информацию.

Чтобы решить, что делать с некорректными и пропущенными значениями, рассмотрим каждый столбец отдельно.

<a name='children'></a>
#### Столбец `children`

In [43]:
print(f'Значения, которые принимает столбец children: {list(data.children.unique())}') #find unique values for column 'children'

Значения, которые принимает столбец children: [1, 0, 3, 2, -1, 4, 20, 5]


In [44]:
data.loc[(data.children == -1), 'children'] = 1 #replace -1 to 1
print(f'Значения, которые теперь принимает столбец children: {list(data.children.unique())}') #check the replacement

Значения, которые теперь принимает столбец children: [1, 0, 3, 2, 4, 20, 5]


В столбце `children` было значение -1. Такого не бывает. Скорее всего это техническая ошибка. Исправили ее заменой на 1.

<a name='dob_years'></a>
#### Столбец `dob_years`

In [45]:
print('Описательная статистика `dob_years`:')
data.dob_years.describe()

Описательная статистика `dob_years`:


count   21,525.00
mean        43.29
std         12.57
min          0.00
25%         33.00
50%         42.00
75%         53.00
max         75.00
Name: dob_years, dtype: float64

Минимальное значения возраста заемщика 0 лет. Посмотрим какие еще нереальные значения могут принимать значения в этом столбце.

In [46]:
print(data[data.dob_years < 14].dob_years.unique()) #are there clients under 14 years old
print(f'{len(data[data.dob_years==0])} строки с нулевым значением возраста заемщика')

[0]
101 строки с нулевым значением возраста заемщика


In [47]:
data.loc[(data.dob_years == 0), 'dob_years'] = data.dob_years.mean() #mean value instead zero
print('Описательная статистика `dob_years` после замены нулевого значения:')
data.dob_years.describe()

Описательная статистика `dob_years` после замены нулевого значения:


count   21,525.00
mean        43.50
std         12.22
min         19.00
25%         34.00
50%         43.00
75%         53.00
max         75.00
Name: dob_years, dtype: float64

Заполнили нули средним значением этого столбца. Описательная статистика столбца изменилась несильно.

<a name='days_employed'></a>
#### Столбец `days_employed`

In [48]:
print('Описательная статистика `days_employed`:')
data.days_employed.describe() #show descriptive statistics

Описательная статистика `days_employed`:


count    19,351.00
mean     63,046.50
std     140,827.31
min     -18,388.95
25%      -2,747.42
50%      -1,203.37
75%        -291.10
max     401,755.40
Name: days_employed, dtype: float64

В столбце `days_employed` содержатся значения, неподходящие для данного столбца: 
 - отрицательные значения,
 - нереально большие - максимальное значение 401755.4 / 247 рабочих дней в году = 1626 лет.
 
Реально возможные значения в этом столбце: от 0 дней  до (60лет*247 рабочих дней в году = 14820)дней.
Посмотрим есть ли такие значения в наших данных.

In [49]:
data[(data.days_employed>=0) & (data.days_employed<=14820)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Таких данных в таблице нет (таблица пустая). Отдельно посмотрим на данные тех строк, где стаж положительный и отдельно где отрицательный.

In [50]:
df_days_empl_neg = data[(data.days_employed<0)] #slice dataframe - only negative days_employed
print('Описательная статистика `days_employed` c отрицательными значениями:')
print(df_days_empl_neg.days_employed.describe())
print(f'Значения, которые принимает столбец income_type при отрицательных значениях days_employed: {df_days_empl_neg.income_type.unique()}')

Описательная статистика `days_employed` c отрицательными значениями:
count    15,906.00
mean     -2,353.02
std       2,304.24
min     -18,388.95
25%      -3,157.48
50%      -1,630.02
75%        -756.37
max         -24.14
Name: days_employed, dtype: float64
Значения, которые принимает столбец income_type при отрицательных значениях days_employed: ['сотрудник' 'компаньон' 'госслужащий' 'студент' 'предприниматель'
 'в декрете']


 Наличие минуса в этих данных, скорее всего, техническая ошибка. Для решения отбросим минус.

In [51]:
df_days_empl_pos = data[(data.days_employed>=0)] #slice dataframe - only positive days_employed
print('Описательная статистика `days_employed` c положительными значениями:')
print(df_days_empl_pos.days_employed.describe())
print(f'Значения, которые принимает столбец income_type при положительных значениях days_employed: {df_days_empl_pos.income_type.unique()}')

Описательная статистика `days_employed` c положительными значениями:
count     3,445.00
mean    365,004.31
std      21,075.02
min     328,728.72
25%     346,639.41
50%     365,213.31
75%     383,246.44
max     401,755.40
Name: days_employed, dtype: float64
Значения, которые принимает столбец income_type при положительных значениях days_employed: ['пенсионер' 'безработный']


Мы видим, что таких данных 3445 строк. Это данные только по пенсионерам и безработным. Все данные с ошибкой(значения нереально большие). Скорее всего, это техническая ошибка. Варианты решения:
 - обратится к составителю данных. Может именно для этих групп тип занятости подсчет трудового стажа дает ошибку.

если нет такой возможности:
 - заменить на медианное значение для данного возраста и пола.
 - отбросить данные по этим типам занятости. Но тогда анализ будет не полноценным.
 
В данной работе будем считать эти данные пропущенными и заменим их на NaN-values. Далее сгруппируем данные по полу и возрасту, а затем для каждой группы найдем медиану и присвоим ее пропускам из той же группы.

In [52]:
data.loc[data['days_employed']>0, 'days_employed'] = np.NaN #NaN values instead possitive
data.loc[data['days_employed']<0, 'days_employed'] = abs(data.days_employed) #absolute value for negative
print('Описательная статистика `days_employed` с nan-values:')
data.days_employed.describe()

Описательная статистика `days_employed` с nan-values:


count   15,906.00
mean     2,353.02
std      2,304.24
min         24.14
25%        756.37
50%      1,630.02
75%      3,157.48
max     18,388.95
Name: days_employed, dtype: float64

Теперь в столбце `days_employed` имеем только положительные данные и nan-values. Пропусков 5619, что достаточно много для того что бы не обращать на них внимания и удалить их. Поэтому логично будет сгруппировать по полу и возрасту, а затем для каждой группы найти медиану и присвоить ее пропускам из той же группы. Если данных для вычисления медианы для какого-то конкретного возраста недостаточно, используем максимальное значение из медианных значений (недостаток данных наблюдается для пожилых клиентов, поскольку для пенсионеров у нас nan-values).

In [53]:
table_days_empl = data.pivot_table(values = 'days_employed', index=['dob_years'], 
                                   columns = 'gender', aggfunc='median') #make pivot table with median value
table_days_empl['M'] = table_days_empl['M'].fillna(table_days_empl['M'].max()) #fill nan-values in pivot_table
def fill_nan_values(row): 
    """
    function return median value of days_employed
    due to gender and age of a client, if there is nan-value of days_employed.
    Else return initial value of days_employed
    """
    if pd.isna(row.days_employed):
        return table_days_empl.loc[row.dob_years, row.gender]
    else:
        return row.days_employed
data['days_employed'] = data.apply(fill_nan_values, axis = 1) #fill nan-values
print('Описательная статистика `days_employed` без nan-values:')
data.days_employed.describe()


Описательная статистика `days_employed` без nan-values:


count   21,525.00
mean     2,391.96
std      2,041.76
min         24.14
25%        984.47
50%      1,991.40
75%      3,019.02
max     18,388.95
Name: days_employed, dtype: float64

<a name='total_income'></a>
#### Столбец `total_income`

In [54]:
pd.options.display.float_format ='{:,.2f}'.format
print('Описательная статистика `total_income`:')
data.total_income.describe()

Описательная статистика `total_income`:


count      19,351.00
mean      167,422.30
std       102,971.57
min        20,667.26
25%       103,053.15
50%       145,017.94
75%       203,435.07
max     2,265,604.03
Name: total_income, dtype: float64

 Имеем пропуски в этом столбце. Аналогично предыдущему столбцу, заполним пропуски медианным значением основываясь на возраст и гендер.

In [55]:
table_total_income = data.pivot_table(values = 'total_income', index=['dob_years'], columns = 'gender', aggfunc='median') #make pivot table with median value
table_total_income['M'] = table_total_income['M'].fillna(table_total_income['M'].mean()) #fill nan-values in pivot_table
def fill_nan_values2(row):
    """
    function return median value of total_income
    due to gender and age of a client, if there is nan-value of total_income.
    Else return initial value of total_income
    """
    if pd.isna(row.total_income):
        return table_total_income.loc[row.dob_years, row.gender]
    else:
        return row.total_income
data['total_income'] = data.apply(fill_nan_values2, axis = 1)
print('Описательная статистика `total_income` после заполнения nan-values:')
data.total_income.describe()

Описательная статистика `total_income` после заполнения nan-values:


count      21,525.00
mean      165,142.81
std        98,065.54
min        20,667.26
25%       107,485.65
50%       143,369.41
75%       195,543.62
max     2,265,604.03
Name: total_income, dtype: float64

<a name='education'></a>
#### Столбец `education` и `education_id`

In [56]:
print(data.education.unique()) 

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


Строковые значения в верхнем и нижнем регистре. Заменим все на нижний.

In [57]:
data['education'] = data.education.str.lower() #convert to lower case
print(data.education.unique()) 

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


In [58]:
print(data.education_id.unique()) 

[0 1 2 3 4]


Теперь количество уникальных значений этих столбцов совпадает.

<a name='family_status'></a>
#### Столбцы `family_status`  и `family_status_id`

In [59]:
print(data.family_status.unique())
print(data.family_status_id.unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
[0 1 2 3 4]


In [60]:
data['family_status'] = data.family_status.str.lower() #convert to lower case
print(data.family_status.unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


Перевели все названия категорий в нижний регистр. для порядка.

<a name='gender'></a>
#### Столбец `gender`

In [61]:
data.gender.value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Как видно из таблицы выше, только один человек имеет этот гендер. Можно убрать этоу строку и тогда у нас будет только 2 уникальных значения гендера. Но можно и оставить, на данном этапе анализа эта строка совсем не мешается.

<a name='income_type'></a>
#### Столбец `income_type`

In [62]:
data.income_type.value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Если для дальнейшего анализа нам потребуется этот столбец, мне кажется, лучше убрать малочисленные категории: безработный, предприниматель, в декрете, студент.

<a name='debt'></a>
#### Столбец `debt `

In [63]:
data.debt.value_counts()

0    19784
1     1741
Name: debt, dtype: int64

8.8% клиентов имеют задолжности по возврату кредитов.

<a name='purpose'></a>
#### Столбец `purpose`

In [64]:
print('Уникальные значения столбца `purpose`:')
print(data.purpose.unique())

Уникальные значения столбца `purpose`:
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образ

Видим, что многие цели получения кредита совпадают, хотя записаны по разному. Видимо у операторов вводивших эти данные не было четких инструкций, как должно это выглядеть в данных. Например: цели - 'cыграть свадьбу', 'на проведение свадьбы', 'свадьба' по сути одно и то же. Необходимо их объединить под одним названием. Для этого используем лемматизацию.

In [65]:
morph = pymorphy2.MorphAnalyzer() #create lemmatizer stopwords list
russian_stopwords = stopwords.words("russian") #create stopwords list

def lemmatize(text): #function of lemmatizing
    words = text.split() # split text to words
    res = []
    for word in words:
        if word not in russian_stopwords: #exclude stopwords
            p = morph.parse(word)[0] #find dictionary form of a word
            res.append(p.normal_form) 
    return res 

lemmas = []
for purpose in data.purpose.unique():
    lemma = lemmatize(purpose)
    lemmas += lemma
print(Counter(lemmas)) #counting the number of mentions of words in purposes

Counter({'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жильё': 7, 'операция': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заняться': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


Используя подсчет числа лемм в целях получения кредита, составим вручную список целей `purpose_list`, подходящих для категоризации.

Далее объединим категории в соответствии с этим списком.

In [66]:
purpose_list = ['образование','свадьба','недвижимость','автомобиль','жильё','строительство','ремонт']

def find_word(text): #function that return word from text if it is in purpose_list
    for word in lemmatize(text):
        if word in purpose_list:
            return word
        
dict = {} #create a dictionary that relates purposes and word in purpose_list
for purpose in data.purpose.unique():
    dict[purpose] = find_word(purpose)
    
data['purpose_cat'] = data['purpose'].map(dict) #create a column based on dictionary values
data.purpose_cat.value_counts()


недвижимость     4486
автомобиль       4315
образование      4022
жильё            3861
свадьба          2348
строительство    1881
ремонт            612
Name: purpose_cat, dtype: int64

<a name='типы данных'></a>
### Замена типов данных

Стоит привести целочисленные значения к типу int, а не оставлять во float, так как int почти повсеместно более эффективен.

In [67]:
data = data.astype({'days_employed':'int', 'dob_years':'int', 'total_income':'int'})
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жильё
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жильё
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,2575,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


Итак, данные приведены в порядок. Проведем теперь нужную категоризацию данных, необходимую для ответов на поставленные задачи.

<a name='категоризация'></a>
### Категоризация данных

Для начала категоризируем столбец `children`. создадин новый столбец `children_cat`, в котором будут 3 категории: нет детей, 1-2 детей и многодетные.

In [68]:
data['children_cat'] = data.children.apply(lambda x: 'нет детей' if x==0 else '1-2 детей' if (x==1)or(x==2) else 'многодетные') #categorizing column `children`

Теперь перейдем к категоризации ежемесячного дохода. Для начала посмотрим описательную статистику этой переменной. Будем проводить категоризацию в соответствии с квартилями. Квартили — значения, которые делят таблицу данных на четыре группы, содержащие приблизительно равное количество наблюдений. Поэтому получим 4 группы заработной платы: 
 - низкая - будет соответствовать данным из 1 квартиля, 
 - ниже среднего - 2 квартиль, 
 - выше среднего - 3 квартиль,
 - высокая - 4 квартиль.

In [69]:
data.total_income.describe()

count      21,525.00
mean      165,142.30
std        98,065.54
min        20,667.00
25%       107,485.00
50%       143,369.00
75%       195,543.00
max     2,265,604.00
Name: total_income, dtype: float64

In [70]:
def categorize_total_income(income): #function for categorizing `total_income`
    if income<=100000:
        return 'низкая'
    elif 100000<income<=150000:
        return 'ниже среднего'
    elif 150000<income<=200000:
        return 'выше среднего'
    else:
        return 'высокая'
    
data['total_income_cat'] = data['total_income'].apply(categorize_total_income)

Данные по семейному положению оставим без изменений.

In [71]:
data.family_status.value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Изменим тип данных для `children_cat` и `total_income_cat`, чтобы данные выводились в определенном порядке.

In [72]:
data = data.astype({'children_cat' : pd.CategoricalDtype(['нет детей', '1-2 детей', 'многодетные'], ordered = True)},
                  {'total_income_cat' : pd.CategoricalDtype(['низкая', 'ниже среднего', 'выше среднего', 'высокая'], ordered = True)})

### **Вывод**

Итак, мы закончили пердобработку данных. 

- Было сделано предположение, что дубликаты не являются испорченными данными, а представляют разных клиентов с одиниковыми характеристиками.

    В [обработке дубликатов](#дубликаты) это подробно описыватся. Поэтому эти данные были оставлены в датасете.
    
- В таблице данных найдены артефакты и пропуски. Для того чтобы исправить данные и заполнить пропуски был рассмотрен каждый столбец в отдельности:
    * для столбцов [`gender`](#gender), [`debt`](#debt), [`income_type`](#income_type), [`family_status`](#family_status) не потребовалась корректировка данных.
    * в столбцах [`children`](#children), [`dob_years`](#dob_years), [`education`](#education)  были найдены и исправлены небольшие недочеты.
    * столбцы [`days_employed`](#days_employed), [`total_income`](#total_income) больше всех нуждались в предобработке, так как именно они содержали множество некорректных данных и пропусков.
    * для столбца [`purpose`](#purpose) потребовалась лемматизация.
    
- Для характеристик, необходимых для дальнейшего анализа в рамках этой работы была проведена [категоризация данных](#категоризация).


Теперь нужно разобраться влияет ли какая-то категория на вероятность задолженности по кредиту и ответить на вопросы сформулированные в начале.

## 3. Проверка гипотез

Для каждого вопроса создадим сводную таблицу с выводом процентов, которые показывают вероятность стать должником у клиента принадлежащего к определенной категоризированной группе.

#### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [73]:
def create_pivot_table(col1, col2 = 'debt'):
    """
    function that return pivot table with percertages
    taking as input 2 columns from dataframe:
        col1 any categorical, col2 = 'debt' by default
    aggregating function = count
    """
    table = data.groupby([col1, col2]).agg({col2: ['count']}) #create pivot table
    table.columns = table.columns.map('_'.join) #convert MultiIndex to Index
    index = table.columns.item()
    table.reset_index()
    table[index+'%'] = table[index]*100/table.groupby(col1)[index].sum() #add percentages
    print(f'Сводная таблица: {col1} и {col2}')
    return table.unstack()

create_pivot_table('children_cat')



Сводная таблица: children_cat и debt


debt_count       debt_count%     
debt                  0     1           0    1
children_cat                                  
нет детей         13086  1063       92.49 7.51
1-2 детей          6281   639       90.77 9.23
многодетные         417    39       91.45 8.55

Клиенты с детьми чаще имеют задолжность по кредитам(9.2% у клиентов имеющих 1-2 детей, 8.6% у многодетных). 

Но нельзя сказать, что чем детей больше, тем вероятность задолжности возрастает.

Наверное, эти данные говорят о том, что клиентам без детей  все таки легче выплачивать кредит, а у многодетных клиентов возрастает ответственность.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [74]:
create_pivot_table('family_status')

Сводная таблица: family_status и debt


debt_count      debt_count%     
debt                           0    1           0    1
family_status                                         
в разводе                   1110   85       92.89 7.11
вдовец / вдова               897   63       93.44 6.56
гражданский брак            3789  388       90.71 9.29
женат / замужем            11449  931       92.48 7.52
не женат / не замужем       2539  274       90.26 9.74

Клиенты, не состоящие в браке, а также клиенты состоящие в гражданском браке наиболее склонны к задержке выплат по кредиту (9.7% и 9.3% имеют задолжности).

Самые ответственные по выплатам - вдовцы (6.5% имеют задолжности).

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [75]:
create_pivot_table('total_income_cat')

Сводная таблица: total_income_cat и debt


debt_count      debt_count%     
debt                      0    1           0    1
total_income_cat                                 
высокая                4708  358       92.93 7.07
выше среднего          4308  436       90.81 9.19
ниже среднего          6658  593       91.82 8.18
низкая                 4110  354       92.07 7.93

Как и ожидалось, клиенты с высоким уровнем дохода наименее склонны с задержке выплат (7% имеют задолжности). 

Но главные нарушители по выплатам  - клиенты с доходом выше среднего(9.2% имеют задолжгости).

#### Как разные цели кредита влияют на его возврат в срок?

In [76]:
create_pivot_table('purpose_cat')

Сводная таблица: purpose_cat и debt


debt_count      debt_count%     
debt                   0    1           0    1
purpose_cat                                   
автомобиль          3912  403       90.66 9.34
жильё               3588  273       92.93 7.07
недвижимость        4156  330       92.64 7.36
образование         3652  370       90.80 9.20
ремонт               577   35       94.28 5.72
свадьба             2162  186       92.08 7.92
строительство       1737  144       92.34 7.66

Больше всего вероятность стать задолжником у клиента, который берёт кредит на автомобиль (9.3%). 

Меньше всего вероятность стать задолжником у берущих в кредит жильё (7%) и у делающих ремонт (5.7%).

## 4. **ВЫВОД**
В рамках данной работы были решены следующие задачи:

- Проведена предобработка данных:
    - найдены аномальные данные и пропущенные значения;
    - разработан, описан и применен подход к их исправлению;
    - проведена категоризация данных, необходимых для анализа;
- Даны ответы на поставленные вопросы.
    
_**Практическая значимость:**_

Предобработка данных, проведенная в данной работе позволяет использовать очищенные данные для дальнейшего анализа и построения моделей кредитного скоринга.
Полученные сводные таблицы позволяют оценить зависимость надежности клиента от некоторых характеристик.

_**Рекомендации по сбору данных:**_

- проверить, как считаются данные столбца `days_employed` для пенсионеров и безработных;
- дать четкие инструкции операторам о записи данных в столбец `purpose`.